In [2]:
import pandas as pd
from rdkit.Chem import PandasTools
from ast import literal_eval



In [ ]:

# df = pd.read_pickle("schembl_summs_v4_final_with_fingerprint_tsne.pkl")
df = pd.read_pickle("schembl_summs_v4_final_with_chemberta_umap.pkl")
df["summarizations"] = df["summarizations"].apply(literal_eval)
PandasTools.AddMoleculeColumnToFrame(df, smilesCol='smiles')


In [3]:
from rdkit.Chem import Draw

df['img'] = df['ROMol'].map(lambda x: Draw.MolToImage(x, size=(200, 200)))

[10:10:30] unsupported number of radical electrons 4
[10:13:31] unsupported number of radical electrons 4
[10:13:43] unsupported number of radical electrons 4
[10:16:01] unsupported number of radical electrons 4


In [4]:
from io import BytesIO

def pil_to_b64(pil):
    with BytesIO() as buffer:
        pil.save(buffer, format="PNG")
        encoded_image = base64.b64encode(buffer.getvalue()).decode("utf-8")
        return "data:image/png;base64, " + encoded_image

In [6]:
import base64


df['im_url'] = df['img'].map(pil_to_b64)

In [8]:
df[["smiles", "cid", "summarizations", "cb_umap_x", "cb_umap_y", "im_url"]].to_pickle("umap_chemberta_plotly.pkl")

In [3]:
# SCRATCH

df_1 = pd.read_pickle("tsne_fingerprint_plotly.pkl")
df_2 = pd.read_pickle("tsne_chemberta_plotly.pkl")
#rename chemberta_tsne_x to cb_tsne_x
df_2 = df_2.rename(columns={"chemberta_tsne_x": "cb_tsne_x", "chemberta_tsne_y": "cb_tsne_y"})
df_3 = pd.read_pickle("umap_chemberta_plotly.pkl")

df_1 = pd.merge(df_1, df_2[["cid", "cb_tsne_x", "cb_tsne_y"]], on="cid")
df_1 = pd.merge(df_1, df_3[["cid", "cb_umap_x", "cb_umap_y"]], on="cid")

df_1.to_pickle("plotly_all.pkl")

In [1]:
import pandas as pd
from rdkit.Chem import PandasTools
from ast import literal_eval

df_labels = pd.read_csv("../../results/schembl_summs_v4_gpt_cleaned_eps_0.340_diff_20030_final.csv")
df_labels = df_labels.drop(columns = ['Unnamed: 0'])


df_umap_tsne = pd.read_pickle("plotly_all.pkl")
df_umap_tsne = df_umap_tsne[["cid", "fp_tsne_x", "fp_tsne_y", "cb_tsne_x", "cb_tsne_y", "cb_umap_x", "cb_umap_y", "im_url"]]

In [2]:
df_final = df_labels.merge(df_umap_tsne, on="cid", how="left")
df_final["summarizations"] = df_final["summarizations"].map(literal_eval).map(set)
df_final.to_pickle("schembl_summs_v4_final_cb_fp_tsne_umap.pkl")

In [3]:
from collections import Counter
import pickle as pkl

counter = Counter()
for i in df_final["summarizations"]:
    counter.update(i)

# create list of strings sorted in order by frequency
sorted_terms = [x[0] for x in counter.most_common()]
with open("sorted_terms.pkl", "wb") as f:
    pkl.dump(sorted_terms, f)


In [4]:
# sorted_terms

In [5]:
df_cb = pd.read_pickle("schembl_summs_v4_final_with_chemberta.pkl")
df_fp = pd.read_pickle("schembl_summs_v4_final_with_fingerprint.pkl")

In [6]:
df_features = df_labels
df_features = df_features.merge(df_cb[["cid", "features"]], on="cid", how="left")
df_features = df_features.merge(df_fp[["cid", "fingerprint"]], on="cid", how="left")
print(len(df_features))
# drop rows with no features
df_features = df_features.dropna()
print(len(df_features))
df_features.to_pickle("../../results/schembl_summs_v4_gpt_cleaned_eps_0.340_diff_20030_final_with_fp_cb.pkl")

99454
99373


In [7]:
df_features["summarizations"] = df_features["summarizations"].map(literal_eval)

In [8]:
import numpy as np


In [9]:
df_features["fingerprint"] = df_features["fingerprint"].apply(np.array)

In [12]:
len(df_features)

99373

In [10]:
df_features[["cid", "summarizations", "features", "fingerprint"]].to_pickle("../../results/schembl_summs_v4_final_with_only_summs_fingerprint_and_chemberta.pkl")